In [1]:
import MDAnalysis as mda
import nglview as nv
import pandas as pd
import numpy as np
import matplotlib as plt
import subprocess
import copy
import math

In [2]:
def find_ss_from_pdb(infile,dsspcode='H'):
    output = subprocess.check_output(['mkdssp', infile])
    out_string=output.decode("utf-8") 
    lsw=False
    new_helix=False

    ss_dict = {} # Dicionnary of secondary structures
    for line in out_string.splitlines():
        #print(line)
        if line.find("#  RESIDUE") > 0:
            lsw=True
            continue
        if lsw:
            chainId=line[11]
            if chainId==' ':
                continue
            resn=int(line[5:10])
            seco=line[16]
            if not (chainId in ss_dict):
                ss_dict[chainId]={}
                helix_id=-1
            if seco==dsspcode and not(new_helix):
                #print(line)
                helix_id+=1
                ss_dict[chainId][helix_id]=[resn]
                new_helix=True
            elif seco!=dsspcode and new_helix:
                ss_dict[chainId][helix_id].append(resn)
                new_helix=False
    return ss_dict
def dot(v,w):
    x,y,z = v
    X,Y,Z = w
    return x*X + y*Y + z*Z

def cross(a, b):
    c = [a[1]*b[2] - a[2]*b[1],
         a[2]*b[0] - a[0]*b[2],
         a[0]*b[1] - a[1]*b[0]]
    return c

def vec_norm(v):
    x,y,z = v
    return math.sqrt(x*x + y*y + z*z)


def py_ang(v1, v2):
   
    cosang = dot(v1, v2)
    sinang = vec_norm(cross(v1, v2))
    return math.atan2(sinang, cosang)


### Analysing geometric CVs for tetrasomes from Bilocapic unwrapping classes

In [3]:
mol=mda.Universe('structures/CrEM_unwr_1class_tetra.pdb','structures/CrEM_unwr_tetra_trj.xtc')
helix_dict=find_ss_from_pdb('structures/CrEM_unwr_1class_tetra.pdb','H')
helix_dict_mda_selections=copy.deepcopy(helix_dict)
helix_dict_directions=copy.deepcopy(helix_dict)

view=nv.show_mdanalysis(mol)
view.update_cartoon(opacity=0.25, component=0)
view.update_representation(opacity=0.25, repr_index=5)
shape=view.shape
for chain_id, chain_dict in helix_dict.items():
    for helix_id, index_list in chain_dict.items():
        selection=mol.select_atoms('segid %s and resid %d:%d and name CA'%(chain_id,*index_list))
        helix_dict_mda_selections[chain_id][helix_id]=selection
        roughth_vec=selection.positions[-1]-selection.positions[0]
        direction=selection.principal_axes()[2]
        #reorient if vector is in counter to ca(start)-ca(stop) vec
        if py_ang(roughth_vec,direction) > 1.5: # ~90 degrees in rad
            direction=-direction
        center=selection.center_of_geometry()
        helix_dict_directions[chain_id][helix_id]=[direction,center]
        
        length=(index_list[1]-index_list[0])*1.5
        start=center-direction*length/2
        stop=center+direction*length/2
        shape.add_sphere(center.tolist(),[1,0,0],1)
        shape.add_label(center.tolist(),[0,0,0],5,'Chain %s,%d'%(chain_id,helix_id))
        shape.add_arrow(start.tolist(),stop.tolist(),[0,1,1],1)
view


NGLWidget(count=4)

In [4]:
angles_df_nucl = pd.DataFrame(columns=['H3-H3 angle','H2A-H2B-1 angle','DNA H2A-H2B-1 angle','DNA torsion - 1',
                                                     'H2A-H2B-2 angle','DNA H2A-H2B-2 angle','DNA torsion - 2','DNA torsion'])
selections={'A':helix_dict_mda_selections['A'][2],
            'E':helix_dict_mda_selections['E'][2],
            'B':helix_dict_mda_selections['B'][1],
            'F':helix_dict_mda_selections['F'][1]}

n1n9_select_str=' ((resname DA DG ADE GUA and name N9) or (resname DC DT CYT THY and name N1))'

contact1_sel=mol.select_atoms('((segid I and resnum -24) or (segid J and resnum  24)) and %s'%n1n9_select_str)
contact2_sel=mol.select_atoms('((segid I and resnum  -7) or (segid J and resnum   7)) and %s'%n1n9_select_str)
contact3_sel=mol.select_atoms('((segid I and resnum   7) or (segid J and resnum  -7)) and %s'%n1n9_select_str)
contact4_sel=mol.select_atoms('((segid I and resnum  24) or (segid J and resnum -24)) and %s'%n1n9_select_str)

for ts in mol.trajectory:
    centers={}
    vectors={}
    for chainID, selection in selections.items():
        centers[chainID]=selection.center_of_geometry()
        roughth_vec=selection.positions[-1]-selection.positions[0]
        direction=selection.principal_axes()[2]
        if py_ang(roughth_vec,direction) > 1.5: # ~90 degrees in rad
                direction=-direction
        vectors[chainID]=direction

    angle_h3_h3=np.rad2deg(py_ang(vectors['A'],vectors['E']))
    angle_h3_h4_A=np.rad2deg(py_ang(vectors['A'],vectors['B']))
    angle_h3_h4_E=np.rad2deg(py_ang(vectors['E'],vectors['F']))
    
    curvature_points={}
    for pos in [-25,-20,-15,-10,-5,5,10,15,20,25]:
        curvature_points[pos]={}
        curvature_points[pos]['sel']=mol.select_atoms('((segid I and resnum %d) or (segid J and resnum  %d)) and %s'%(pos,-pos,n1n9_select_str))
        curvature_points[pos]['coord']=np.average(curvature_points[pos]['sel'].positions,axis=0)
    DNA_angle_h3_h4_A=np.rad2deg(mda.lib.distances.calc_angles(curvature_points[-25]['coord'],curvature_points[-15]['coord'],
                                                               curvature_points[-5]['coord']))
    DNA_angle_h3_h4_E=np.rad2deg(mda.lib.distances.calc_angles(curvature_points[25]['coord'],curvature_points[15]['coord'],
                                                               curvature_points[5]['coord']))
    
    contact1=np.average(contact1_sel.positions,axis=0)
    contact2=np.average(contact2_sel.positions,axis=0)
    contact3=np.average(contact3_sel.positions,axis=0)
    contact4=np.average(contact4_sel.positions,axis=0)
    DNA_dihe=np.rad2deg(mda.lib.distances.calc_dihedrals(contact1,contact2,contact3,contact4))
    
    DNA_dihe_A=np.rad2deg(mda.lib.distances.calc_dihedrals(curvature_points[-25]['coord'],curvature_points[-20]['coord'],
                                                           curvature_points[-10]['coord'],curvature_points[-5]['coord']))
    DNA_dihe_E=np.rad2deg(mda.lib.distances.calc_dihedrals(curvature_points[25]['coord'],curvature_points[20]['coord'],
                                                           curvature_points[10]['coord'],curvature_points[5]['coord']))
    
    angles_df_nucl.loc[ts.time] =[angle_h3_h3, angle_h3_h4_A, DNA_angle_h3_h4_A,DNA_dihe_A,
                                               angle_h3_h4_E, DNA_angle_h3_h4_E, DNA_dihe_E, DNA_dihe]
angles_df_nucl

,H3-H3 angle,H2A-H2B-1 angle,DNA H2A-H2B-1 angle,DNA torsion - 1,H2A-H2B-2 angle,DNA H2A-H2B-2 angle,DNA torsion - 2,DNA torsion
0.0,77.489829,141.844099,129.772158,-7.790299,140.899388,129.085776,-4.406519,-13.580784
1.0,82.834653,140.502331,131.906647,-5.806249,139.023654,127.434458,-6.139738,-15.173990
2.0,79.580777,140.508373,128.972616,-6.732410,142.294292,127.484060,-3.854885,-15.242811
3.0,86.717354,138.587392,136.130641,-26.897788,135.515947,130.492826,-3.778868,-17.958307


### Analysing geometric CVs for tetrasomes from Bilocapic translocation classes

In [5]:
mol=mda.Universe('structures/CrEM_transloc_1class_tetra.pdb','structures/CrEM_transloc_tetra_trj.xtc')
helix_dict=find_ss_from_pdb('structures/CrEM_transloc_1class_tetra.pdb','H')
helix_dict_mda_selections=copy.deepcopy(helix_dict)
helix_dict_directions=copy.deepcopy(helix_dict)

view=nv.show_mdanalysis(mol)
view.update_cartoon(opacity=0.25, component=0)
view.update_representation(opacity=0.25, repr_index=5)
shape=view.shape
for chain_id, chain_dict in helix_dict.items():
    for helix_id, index_list in chain_dict.items():
        selection=mol.select_atoms('segid %s and resid %d:%d and name CA'%(chain_id,*index_list))
        helix_dict_mda_selections[chain_id][helix_id]=selection
        roughth_vec=selection.positions[-1]-selection.positions[0]
        direction=selection.principal_axes()[2]
        #reorient if vector is in counter to ca(start)-ca(stop) vec
        if py_ang(roughth_vec,direction) > 1.5: # ~90 degrees in rad
            direction=-direction
        center=selection.center_of_geometry()
        helix_dict_directions[chain_id][helix_id]=[direction,center]
        
        length=(index_list[1]-index_list[0])*1.5
        start=center-direction*length/2
        stop=center+direction*length/2
        shape.add_sphere(center.tolist(),[1,0,0],1)
        shape.add_label(center.tolist(),[0,0,0],5,'Chain %s,%d'%(chain_id,helix_id))
        shape.add_arrow(start.tolist(),stop.tolist(),[0,1,1],1)
view

NGLWidget(count=3)

In [6]:
angles_df_nucl = pd.DataFrame(columns=['H3-H3 angle','H2A-H2B-1 angle','DNA H2A-H2B-1 angle','DNA torsion - 1',
                                                     'H2A-H2B-2 angle','DNA H2A-H2B-2 angle','DNA torsion - 2','DNA torsion'])
selections={'A':helix_dict_mda_selections['A'][2],
            'E':helix_dict_mda_selections['E'][2],
            'B':helix_dict_mda_selections['B'][1],
            'F':helix_dict_mda_selections['F'][1]}

n1n9_select_str=' ((resname DA DG ADE GUA and name N9) or (resname DC DT CYT THY and name N1))'

contact1_sel=mol.select_atoms('((segid I and resnum -24) or (segid J and resnum  24)) and %s'%n1n9_select_str)
contact2_sel=mol.select_atoms('((segid I and resnum  -7) or (segid J and resnum   7)) and %s'%n1n9_select_str)
contact3_sel=mol.select_atoms('((segid I and resnum   7) or (segid J and resnum  -7)) and %s'%n1n9_select_str)
contact4_sel=mol.select_atoms('((segid I and resnum  24) or (segid J and resnum -24)) and %s'%n1n9_select_str)

for ts in mol.trajectory:
    centers={}
    vectors={}
    for chainID, selection in selections.items():
        centers[chainID]=selection.center_of_geometry()
        roughth_vec=selection.positions[-1]-selection.positions[0]
        direction=selection.principal_axes()[2]
        if py_ang(roughth_vec,direction) > 1.5: # ~90 degrees in rad
                direction=-direction
        vectors[chainID]=direction

    angle_h3_h3=np.rad2deg(py_ang(vectors['A'],vectors['E']))
    angle_h3_h4_A=np.rad2deg(py_ang(vectors['A'],vectors['B']))
    angle_h3_h4_E=np.rad2deg(py_ang(vectors['E'],vectors['F']))
    
    curvature_points={}
    for pos in [-25,-20,-15,-10,-5,5,10,15,20,25]:
        curvature_points[pos]={}
        curvature_points[pos]['sel']=mol.select_atoms('((segid I and resnum %d) or (segid J and resnum  %d)) and %s'%(pos,-pos,n1n9_select_str))
        curvature_points[pos]['coord']=np.average(curvature_points[pos]['sel'].positions,axis=0)
    DNA_angle_h3_h4_A=np.rad2deg(mda.lib.distances.calc_angles(curvature_points[-25]['coord'],curvature_points[-15]['coord'],
                                                               curvature_points[-5]['coord']))
    DNA_angle_h3_h4_E=np.rad2deg(mda.lib.distances.calc_angles(curvature_points[25]['coord'],curvature_points[15]['coord'],
                                                               curvature_points[5]['coord']))
    
    contact1=np.average(contact1_sel.positions,axis=0)
    contact2=np.average(contact2_sel.positions,axis=0)
    contact3=np.average(contact3_sel.positions,axis=0)
    contact4=np.average(contact4_sel.positions,axis=0)
    DNA_dihe=np.rad2deg(mda.lib.distances.calc_dihedrals(contact1,contact2,contact3,contact4))
    
    DNA_dihe_A=np.rad2deg(mda.lib.distances.calc_dihedrals(curvature_points[-25]['coord'],curvature_points[-20]['coord'],
                                                           curvature_points[-10]['coord'],curvature_points[-5]['coord']))
    DNA_dihe_E=np.rad2deg(mda.lib.distances.calc_dihedrals(curvature_points[25]['coord'],curvature_points[20]['coord'],
                                                           curvature_points[10]['coord'],curvature_points[5]['coord']))
    
    angles_df_nucl.loc[ts.time] =[angle_h3_h3, angle_h3_h4_A, DNA_angle_h3_h4_A,DNA_dihe_A,
                                               angle_h3_h4_E, DNA_angle_h3_h4_E, DNA_dihe_E, DNA_dihe]
angles_df_nucl

,H3-H3 angle,H2A-H2B-1 angle,DNA H2A-H2B-1 angle,DNA torsion - 1,H2A-H2B-2 angle,DNA H2A-H2B-2 angle,DNA torsion - 2,DNA torsion
0.0,78.228871,141.534818,130.252741,-4.518642,140.748759,127.861953,-5.892770,-14.647480
1.0,83.974267,138.766832,131.109287,-11.820698,138.148785,129.909996,-11.879835,-14.165973
2.0,85.991230,137.017722,130.180291,-8.196721,138.489280,132.637077,-7.414119,-11.778157
